In [4]:
import time

import pickle
import pandas as pd

from tqdm import tqdm
import numpy as np

In [7]:
def getFiltered_csv(name):
    # read tsv
#     tsv = pd.read_csv('./for_biobert/body_text_1_test.tsv', sep='\t')
    # read result_csv
    result_csv = pd.read_csv('./'+ name +'_test_result.csv',encoding='utf-8')
    # read labeled_csv
    labeled_csv = pd.read_csv('./labeled_'+ name +'.json.csv',encoding='utf-8')
    if(len(labeled_csv) == len(result_csv)):
        labeled_csv['biobert'] = result_csv['label']
    else:
        print('len not same')
    return labeled_csv[labeled_csv['biobert']==True]

In [23]:
abstract = getFiltered_csv('abstract')

In [24]:
body = []
for i in tqdm(range(11)):
    body.append(getFiltered_csv('body_text_'+str(i)))

100%|██████████| 11/11 [08:33<00:00, 46.66s/it]


In [25]:
for i in tqdm(range(11)):
    abstract = abstract.append(body[i])

100%|██████████| 11/11 [00:56<00:00,  5.17s/it]


In [26]:
abstract.to_csv('./filter_data.csv' , index=False)

In [27]:
all_len = len(abstract)

In [28]:
all_len

61907784

In [29]:
def getLen(name):
    # read tsv
#     tsv = pd.read_csv('./for_biobert/body_text_1_test.tsv', sep='\t')
    # read result_csv
    result_csv = pd.read_csv('./'+ name +'_test_result.csv',encoding='utf-8')
    # read labeled_csv
    return len(result_csv)

In [30]:
alll = 0
for i in tqdm(range(11)):
    alll = alll + getLen('body_text_'+str(i))

100%|██████████| 11/11 [00:58<00:00,  5.36s/it]


In [31]:
alll = alll + getLen('abstract')

In [32]:
alll

95718592

In [33]:
abstract

,index,subject,relation,object,sentence,sent,label,biobert
0,0,Zuyderland Medical Centre,is secondary care hospital in,Netherlands,SETTING: Two EDs of the Zuyderland Medical Cen...,SETTING: Two EDs of the @ZUYDERLAND MEDICAL CE...,1,True
1,1,Inclusion criteria,were admission to,hospital,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True
2,2,Inclusion criteria,were admission with symptoms suggestive of,COVID-19,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True
3,3,Inclusion criteria,were admission with symptoms positive result of,PCR,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True
4,4,Inclusion criteria,were,admission,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) @ADMISSION# to t...,1,True
...,...,...,...,...,...,...,...,...
738476,738476,specific therapeutic measures,were taken for QUANT_O_1 type of,SCD,"Besides, data were retrospectively affected wh...","Besides, data were retrospectively affected wh...",1,True
738482,738482,findings,indicate that,fever are most common reasons for hospital adm...,"Our findings indicate that VOC, fever, and ACS...","Our @FINDINGS# indicate that VOC, fever, and A...",0,True
738483,738483,findings,indicate that,ACS are most common reasons for hospital admis...,"Our findings indicate that VOC, fever, and ACS...","Our @FINDINGS# indicate that VOC, fever, and A...",0,True
738490,738490,Asthma,was,most prevalent comorbidity,"Asthma was the most prevalent comorbidity, and...","@ASTHMA# was the @MOST PREVALENT COMORBIDITY#,...",1,True


In [39]:
def create_entity(df):
    set_1 = set(df['subject'])
    set_2 = set(df['object'])
    set_entityies = list(set_1) + list(set_2 - set_1)
    results = pd.DataFrame(list(set_entityies))
    results.columns = ["entity_name"]
    sorted_df = results.sort_values("entity_name")
    sorted_df['entity_id'] = range(1, len(sorted_df) + 1)
    order = ['entity_id','entity_name']
    sorted_df = sorted_df[order]
    return sorted_df

In [40]:
entity = create_entity(abstract)

In [41]:
entity

,entity_id,entity_name
17847427,1,!!
17488992,2,!!!
10166307,3,#
21059078,4,# 's
13093296,5,# 0037-17-HMO
...,...,...
6286977,22746520,"𝜗rp ,2"
903010,22746521,𝜗x
11487122,22746522,𝜙
17454303,22746523,𝝳


In [42]:
entity.to_csv('entity.csv', index=False)

In [43]:
relation = abstract

In [45]:
entity_dic = {}
for i in tqdm(range(len(entity.index))):
    row = entity.iloc[i]
    entity_dic[row['entity_name']] = row['entity_id']
relation['subject_id'] = relation.apply(lambda row: entity_dic[row['subject']],axis=1)
relation['object_id'] = relation.apply(lambda row: entity_dic[row['object']],axis=1)

100%|██████████| 22746524/22746524 [53:14<00:00, 7121.14it/s]  


In [47]:
relation=relation.drop('index',axis=1)
relation.columns = ['subject','relation','object','sentence','labeled_sentence','label','biobert','subject_id:START_ID','object_id:END_ID']

In [49]:
order = ['subject_id:START_ID','subject','relation','object_id:END_ID','object','sentence','labeled_sentence','label','biobert']
relation = relation[order]

In [52]:
entity.columns = ['entity_id:ID', 'entity_name']

In [53]:
relation['relation_type:TYPE'] = 'minie-biobert'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
entity

,entity_id:ID,entity_name
17847427,1,!!
17488992,2,!!!
10166307,3,#
21059078,4,# 's
13093296,5,# 0037-17-HMO
...,...,...
6286977,22746520,"𝜗rp ,2"
903010,22746521,𝜗x
11487122,22746522,𝜙
17454303,22746523,𝝳


In [54]:
relation

,subject_id:START_ID,subject,relation,object_id:END_ID,object,sentence,labeled_sentence,label,biobert,relation_type:TYPE
0,5689092,Zuyderland Medical Centre,is secondary care hospital in,3664148,Netherlands,SETTING: Two EDs of the Zuyderland Medical Cen...,SETTING: Two EDs of the @ZUYDERLAND MEDICAL CE...,1,True,minie-biobert
1,2735032,Inclusion criteria,were admission to,12091204,hospital,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True,minie-biobert
2,2735032,Inclusion criteria,were admission with symptoms suggestive of,1019331,COVID-19,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True,minie-biobert
3,2735032,Inclusion criteria,were admission with symptoms positive result of,3910999,PCR,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) admission to the...,1,True,minie-biobert
4,2735032,Inclusion criteria,were,6163377,admission,Inclusion criteria were (1) admission to the h...,@INCLUSION CRITERIA# were (1) @ADMISSION# to t...,1,True,minie-biobert
...,...,...,...,...,...,...,...,...,...,...
738476,19999092,specific therapeutic measures,were taken for QUANT_O_1 type of,4641186,SCD,"Besides, data were retrospectively affected wh...","Besides, data were retrospectively affected wh...",1,True,minie-biobert
738482,10886662,findings,indicate that,10831134,fever are most common reasons for hospital adm...,"Our findings indicate that VOC, fever, and ACS...","Our @FINDINGS# indicate that VOC, fever, and A...",0,True,minie-biobert
738483,10886662,findings,indicate that,336115,ACS are most common reasons for hospital admis...,"Our findings indicate that VOC, fever, and ACS...","Our @FINDINGS# indicate that VOC, fever, and A...",0,True,minie-biobert
738490,642825,Asthma,was,15146503,most prevalent comorbidity,"Asthma was the most prevalent comorbidity, and...","@ASTHMA# was the @MOST PREVALENT COMORBIDITY#,...",1,True,minie-biobert


In [56]:
entity.to_csv('entity.csv', index=False)
relation.to_csv('relation.csv', index=False)